In [ ]:
pip install python-dotenv

In [ ]:
# import modules
import discord
import os
from neuralintents.assistants import BasicAssistant

# from neuralintents import BasicAssistant
import nest_asyncio
import requests
import json
import random
import numpy as np
import pandas as pd

nest_asyncio.apply()

In [ ]:
# Token of the chat-bot for chat-bot identification, get from https://discord.com/developers/applications/1093297147845300337/bot
TOKEN = "MTE2NjYzNTczNjA5NTc5NzMyMQ.G2hPjM.PvTVKR7uoJWoAHRHjG5P6iS82FzJdGy4Z21qDg"

client = discord.Client(intents=discord.Intents.all())
# client = discord.Client(intents=discord.Intents.default())